Notebook I did for an internship, the goal was to recreate the SimCLR workframe on my own and understand how it works. The results are not very satisfying probably because of the loss function being hard to implement efficiently and test

### Installs

In [ ]:
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir
!pip install keras --upgrade
!pip install -Uqq fastbook
!pip install -q tf-nightly-2.0-preview
!pip install XlsxWriter
!pip install git+https://www.github.com/keras-team/keras-contrib.git




  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-ku73v0x2
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-ku73v0x2
     |████████████████████████████████| 47.6 MB 1.4 MB/s 
  Created wheel for albumentations: filename=albumentations-1.1.0-py3-none-any.whl size=102176 sha256=a19dd2a7cd6175732ead48209b20a5fb37b22e3284c9f9b8852665689f71fa7a
  Stored in directory: /tmp/pip-ephem-wheel-cache-4nqyxozq/wheels/63/11/1a/c77caf3ae9b9b6d57b3ee5e6a41a50f3bc12c66a70f6b90bf0
Successfully built albumentations
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


     |████████████████████████████████| 1.3 MB 13.9 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0


     |████████████████████████████████| 720 kB 13.5 MB/s 
     |████████████████████████████████| 46 kB 4.7 MB/s 
     |████████████████████████████████| 189 kB 45.7 MB/s 
     |████████████████████████████████| 1.2 MB 35.7 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
     |████████████████████████████████| 51 kB 326 kB/s 
ERROR: Could not find a version that satisfies the requirement tf-nightly-2.0-preview (from versions: none)
ERROR: No matching distribution found for tf-nightly-2.0-preview
     |████████████████████████████████| 149 kB 11.1 MB/s 
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-eaonwfuo
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-eaonwfuo
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=d5d8ac7778d54a2a55b19fb6916acc059d286a633b366c8f986ed0e608f68321
  Stored in directory: /tmp/pip-ephem-wheel-cache-hzvyn09q

## Imports

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tfc
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical # The neural networks are made and trained using tensorflow
from tensorflow import one_hot

import tensorflow_datasets as tfds # To manipulate datasets

import albumentations as alb # For data augmentation

import xlsxwriter as xw
from xlsxwriter.utility import xl_rowcol_to_cell
import pandas as pd # To make xls file recording trainings


# from keras_contrib.optimizers import LARS 
# This optimiser is not callable in eager execution

import numpy as np
from math import log, exp, ceil
import matplotlib.pyplot as plt # Math & plot


from datetime import datetime, timezone, timedelta
import pytz
now = datetime.now()
tz = pytz.timezone("Europe/Paris") # To add metadata to the xls test sheets

from time import sleep, time # To test the speed of my functions

from tqdm.notebook import tqdm, tqdm_notebook, trange # To add loading bars to the training loop, it's very usefull since the training time can be quite long and varies a lot

import os 
import fastbook as fb# Save/load xls files and neural networks


fb.setup_book()

date = now.astimezone(tz).strftime("%d-%m-%Y")
PATH = str(fb.gdrive)+"/"+date+"/"
if not os.path.exists(PATH):
  os.mkdir(PATH)

#Main code

## Import Data

On the final version we're using the cifar10 dataset as tensorflow_dataset's datasets. I tried several module to manipulate datasets and this one seems the best I found for what I needed. Ineed, I needed a precise manipulation of the individual elements of the dataset since the contrastive learning require a pairing of each augmentation.

In [ ]:
def len_ds(ds : tf.data.Dataset) -> int :
  """
  Compute the length of a dataset, most often the number of batches if the dataset is correctly built.
  This function is computationnaly expensive and should only be used for debugging
  
  Args :
    ds : A dataset

  Return : 
    The lenght of the dataset

  """
  out = 0
  for _ in ds :
    out += 1
  return out

def plot_batch(batch_x : tf.Tensor, title : str = "") :
  """ Plot a batch
  Plot every images in a batch  in 4 columns

  Args :
    batch_x : a batch / set of images to plot
    title : The title of the plot
  
  Returns :
    Nothing
  """
  columns = 4
  rows = len(batch_x)//4 +1
  fig = plt.figure(figsize=(30, 30))
  plt.title(title)
  for i, img in enumerate(batch_x) :
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(img)

def format_image(image : tf.Tensor, label : tf.Tensor = None) -> tf.Tensor :
  """ Format an image
  Transform an image in format [0,255] to a format [0,1] 
  making it easier to process in machine learning

  Args : 
    image : The image to format
  
  Returns : 
    formated image
  """
  if label == None :
    return image/255
  else :
    return image/255, label

def normalize_cifar10_element(image : tf.Tensor, label : tf.Tensor = None) -> tf.Tensor :
  """ Normalize an image from cifar10
  Normalize an image from cifar10 according to the mean and standard deviation of the cifar10 dataset

  Args :
    image : The image to normalize

  returns :
    normalized image 
  """
  if label == None :
    return (image -(0.4914, 0.4822, 0.4465))/(0.247, 0.243, 0.261)
  else :
    return (image -(0.4914, 0.4822, 0.4465))/(0.247, 0.243, 0.261),label

def format_batch(batchx : tf.Tensor, batchy : tf.Tensor = None):
  """ Format a batch
  Format a batch using format image
  can be used with the features tensor or a batch containing features and labels

  Args : 
    batchx : Features
    batchy : Labels

  returns :
    formated batch
  """
  if batchy == None :
    return tf.map_fn(format_image, batchx, fn_output_signature= tf.float32)
  else :
    return tf.map_fn(format_image, batchx, fn_output_signature= tf.float32), batchy

def get_x( x : tf.Tensor, y : tf.Tensor) -> tf.Tensor:
  """ get features from a batch

  Args : 
    x : Features
    y : Labels
  
  Returns :
    Features
  """
  return x

def get_y( x : tf.Tensor, y : tf.Tensor) -> tf.Tensor:
  """ get labels from a batch

  Args : 
    x : Features
    y : Labels
  
  Returns :
    Labels
  """
  return y

def shape(ds : tf.data.Dataset) -> tf.Tensor :
  """ Shape of a dataset

  Args : 
    ds : Dataset

  Returns :
    the shape
  """
  dataset_to_numpy = list(ds.as_numpy_iterator())
  shape = tf.shape(dataset_to_numpy)
  return shape

def from_int_to_categorical(x,y, num_classes) :
  """ Convert labels from a batch from numerical to categorical
  
  Example : 
    a labels tensor [0,2,1] with num_classes = 3 will become 
    [[1,0,0],
     [0,0,1],
     [0,1,0]]
    
    Args : 
      x : Features
      y : Labels
      num_classes : The number of classes

    Returns : 
      A batch with labels formated
  """
  return x, one_hot(y,num_classes)





In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
num_classes = 10
batch_size = 50

def generate_datasets(bank = "cifar10", num_classes = 10) :
  """ Generate and format the datasets needed for SimCLR 
  SimCLR is a semi-supervised learning method, so we have a big unlabeled dataset and a small labeled dataset
  the small labeled dataset is split in three parts respectively the training dataset, the test datatest and the validation datatest

  Args : 
    bank : The name of the dataset, it must be a dataset that can be loaded by tensorflow_datasets
    num_classes : The number of classes in the dataset
  
  Returns :
    Unlabeled dataset
    Train dataset
    Test dataset
    Validation dataset
  
  """
  # Load the datasets
  unlabeled_ds = tfds.load(bank, split='train', as_supervised=True)
  train_ds, test_ds, validation_ds = tfds.load(bank, split=['test[0%:80%]','test[80%:90%]','test[90%:100%]'],as_supervised=True, batch_size=batch_size)
  # Get rid of the labels for the unlabeled dataset, format and prefect the dataset
  unlabeled_ds = unlabeled_ds.map(get_x)
  unlabeled_ds = unlabeled_ds.map(format_image, num_parallel_calls=AUTOTUNE)
  unlabeled_ds = unlabeled_ds.prefetch(AUTOTUNE)
  # Format the labeled datasets
  train_ds = train_ds.map(format_image,  num_parallel_calls=AUTOTUNE)
  test_ds = test_ds.map(format_image,  num_parallel_calls=AUTOTUNE)
  validation_ds = validation_ds.map(format_image,  num_parallel_calls=AUTOTUNE)

  # Normalize the labeled datasets
  train_ds = train_ds.map(normalize_cifar10_element,  num_parallel_calls=AUTOTUNE)
  test_ds = test_ds.map(normalize_cifar10_element,  num_parallel_calls=AUTOTUNE)
  validation_ds = validation_ds.map(normalize_cifar10_element,  num_parallel_calls=AUTOTUNE)

  # Format the labeled datasets
  train_ds = train_ds.map(lambda x,y: from_int_to_categorical(x,y,num_classes))
  test_ds = test_ds.map(lambda x,y: from_int_to_categorical(x,y,num_classes))
  validation_ds = validation_ds.map(lambda x,y: from_int_to_categorical(x,y,num_classes))

  # Prefecth the labeled datasets
  train_ds = train_ds.prefetch(AUTOTUNE)
  test_ds = test_ds.prefetch(AUTOTUNE)
  validation_ds = validation_ds.prefetch(AUTOTUNE)

  return unlabeled_ds, train_ds, test_ds, validation_ds


#def get_generators()

## Pre-build models

The architecture of the neural networks used in this project is not an object.

For this reason I chose to have a simple and fast to train neural network in a first time and then to import a ResNet50 as mentionned in the SimCLR paper for the performance tests. I also imported another neural network, the WideNet to try if that would change the performances.

Another thing to note is that the neural networks are separated in two parts since a projection head is attached to the nn during the training and is swapped for a classifier during the testing.

### Simple Models

In [ ]:
def build_simple_conv() :
  """
  Build a simple convolutive neural network, it needs a classifier to work 
  
  Returns :
    Convolutive body
  """
  model = Sequential(name = "simple_conv_network")

  model.add(Conv2D(64,(3,3),input_shape = (32,32,3), activation='relu', dtype='float32', name='input_image',kernel_initializer='random_normal',bias_initializer='zeros'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Conv2D(128,(3,3), activation='relu', padding='same',kernel_initializer='random_normal',bias_initializer='zeros'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Conv2D(256,(3,3), activation='relu', padding='same',kernel_initializer='random_normal',bias_initializer='zeros'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Flatten())

  return model



def build_resnet():
  """
  Build a resnet from keras
  """
  kerasResNet50 = keras.applications.ResNet50(include_top=False,
                                        weights='imagenet',
                                      input_tensor=None,
                                      input_shape=(32,32,3),
                                      pooling = 'max',
                                      classes = num_classes)

  return kerasResNet50

def build_projection_head():
  """
  Build a projection head
  """
  projection_head = Sequential([
                            Dense(128, activation='relu',kernel_initializer='random_normal',bias_initializer='zeros'),
                            Dense(128, activation='relu',kernel_initializer='random_normal',bias_initializer='zeros')
  ],name = "Projection_head")
  return projection_head



def build_classifier():
  """
  Build a linear classifier
  """
  classifier = Sequential([
                            #Dense(2048, activation='relu'),
                            Dense(num_classes, activation='softmax')
  ],name = "linear_classifier")
  return classifier

### WideNet


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from keras.layers import Conv2D, Dense, Input, add, Activation, Flatten, AveragePooling2D
from keras.callbacks import LearningRateScheduler, TensorBoard
from keras.regularizers import l2
from keras import optimizers
from keras.models import Model

DEPTH              = 28
WIDE               = 2
IN_FILTERS         = 16

CLASS_NUM          = 10
IMG_ROWS, IMG_COLS = 32, 32
IMG_CHANNELS       = 3

BATCH_SIZE         = 128
EPOCHS             = 200
ITERATIONS         = 50000 // BATCH_SIZE + 1
WEIGHT_DECAY       = 0.0005
LOG_FILE_PATH      = './w_resnet/'


def scheduler(epoch):
    if epoch < 60:
        return 0.1
    if epoch < 120:
        return 0.02
    if epoch < 160:
        return 0.004
    return 0.0008

def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.3, 123.0, 113.9]
    std  = [63.0,  62.1,  66.7]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]

    return x_train, x_test

def wide_residual_network(img_input,classes_num,depth,k):
    print('Wide-Resnet %dx%d' %(depth, k))
    n_filters  = [16, 16*k, 32*k, 64*k]
    n_stack    = (depth - 4) // 6

    def conv3x3(x,filters):
        return Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1), padding='same',
        kernel_initializer='he_normal',
        kernel_regularizer=l2(WEIGHT_DECAY),
        use_bias=False)(x)

    def bn_relu(x):
        x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
        x = Activation('relu')(x)
        return x

    def residual_block(x,out_filters,increase=False):
        global IN_FILTERS
        stride = (1,1)
        if increase:
            stride = (2,2)
            
        o1 = bn_relu(x)
        
        conv_1 = Conv2D(out_filters,
            kernel_size=(3,3),strides=stride,padding='same',
            kernel_initializer='he_normal',
            kernel_regularizer=l2(WEIGHT_DECAY),
            use_bias=False)(o1)

        o2 = bn_relu(conv_1)
        
        conv_2 = Conv2D(out_filters, 
            kernel_size=(3,3), strides=(1,1), padding='same',
            kernel_initializer='he_normal',
            kernel_regularizer=l2(WEIGHT_DECAY),
            use_bias=False)(o2)
        if increase or IN_FILTERS != out_filters:
            proj = Conv2D(out_filters,
                                kernel_size=(1,1),strides=stride,padding='same',
                                kernel_initializer='he_normal',
                                kernel_regularizer=l2(WEIGHT_DECAY),
                                use_bias=False)(o1)
            block = add([conv_2, proj])
        else:
            block = add([conv_2,x])
        return block

    def wide_residual_layer(x,out_filters,increase=False):
        global IN_FILTERS
        x = residual_block(x,out_filters,increase)
        IN_FILTERS = out_filters
        for _ in range(1,int(n_stack)):
            x = residual_block(x,out_filters)
        return x


    x = conv3x3(img_input,n_filters[0])
    x = wide_residual_layer(x,n_filters[1])
    x = wide_residual_layer(x,n_filters[2],increase=True)
    x = wide_residual_layer(x,n_filters[3],increase=True)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = AveragePooling2D((8,8))(x)
    x = Flatten()(x)
    #x = Dense(classes_num,
    #    activation='softmax',
    #    kernel_initializer='he_normal',
    #    kernel_regularizer=l2(WEIGHT_DECAY),
    #    use_bias=False)(x)
    return x

def build_WideNet(depth = 28,k = 2) :
  img_input = Input(shape=(32,32,3))
  output = wide_residual_network(img_input,10,depth,k)
  model = Model(img_input, output)
  return model

## Custom loss function

### Own contrastive loss
Contrastive loss according to the formula given in the SimCLR paper

In [ ]:
@tf.function
def contrastive_loss(proj,N, temperature = 0.5, additional_losses = False):
  """
  NT-Xent (the normalized temperature-scaled cross entropy loss)
  Contrastive loss according to the SimCLR scientific papper

  Args : 
    proj : the outputs of the projection head
    indexes : An index matrix needed to compute the loss, it is given by get_indexes but passed as an argument to minimize computation since it only depends of the shape of the features
    temperature : temperature of the loss function
                  it influences the performance of the learning but overall 0.5 seems to be a reliable value

  Returns : 
    The contrastive loss 
  """
  states = tf.math.l2_normalize(proj, -1)
  scores = tf.matmul(states, states, transpose_b=True)  # (bsz, bsz) #Matrice des s(i,j)
  id = tf.eye(scores.shape[0])
  scores = tf.math.subtract(scores,id)
  num = tf.math.divide(scores,temperature)
  indexes = tf.constant(get_indexes_l(int(N/2)))


  l = tf.math.subtract( tf.math.reduce_logsumexp(num, axis = 1), num) # Matrice des l(i,j) 
  loss_dist_match = tf.reduce_mean(tf.gather_nd(l, indexes))
  loss_dist_match = tf.scalar_mul(tf.constant(1/(2*N)),loss_dist_match)

  if additional_losses :
    loss_align = tf.scalar_mul(tf.constant(1/(4*N)),tf.reduce_mean((proj[::2] - proj[1::2])**2))
    return loss_dist_match, loss_align
  return loss_dist_match


def get_indexes_l(n):
  """ Indexes needed for the contrastive loss
  
  Args : 
    n : The batch size / 2

  Returns :
    The indexes needed for contrastive loss 
  """
  id1 = [[2*k+1,2*k] for k in range(n)]
  id2 = [[2*k,2*k+1] for k in range(n)]
  return id1 + id2

### Gitgub Contrastive loss
Contrastive loss given in the official github of the SimCLR paper

In [ ]:
def github_contrastive_loss(hidden,
                         hidden_norm=True,
                         temperature=1.0,
                         tpu_context=None,
                         weights=1.0):
  """Compute loss for model.
  Args:
    hidden: hidden vector (`Tensor`) of shape (2 * bsz, dim).
    hidden_norm: whether or not to use normalization on the hidden vector.
    temperature: a `floating` number for temperature scaling.
    tpu_context: context information for tpu.
    weights: a weighting number or vector.
  Returns:
    A loss scalar.
    The logits for contrastive prediction task.
    The labels for contrastive prediction task.
  """

  LARGE_NUM = 1e9

  # Get (normalized) hidden1 and hidden2.
  if hidden_norm:
    hidden = tf.math.l2_normalize(hidden, -1)
  hidden1, hidden2 = tf.split(hidden, 2, 0)
  batch_size = tf.shape(hidden1)[0]

  # Gather hidden1/hidden2 across replicas and create local labels.
  if tpu_context is not None:
    hidden1_large = tpu_cross_replica_concat(hidden1, tpu_context)
    hidden2_large = tpu_cross_replica_concat(hidden2, tpu_context)
    enlarged_batch_size = tf.shape(hidden1_large)[0]
    # TODO(iamtingchen): more elegant way to convert u32 to s32 for replica_id.
    replica_id = tf.cast(tf.cast(xla.replica_id(), tf.uint32), tf.int32)
    labels_idx = tf.range(batch_size) + replica_id * batch_size
    labels = tf.one_hot(labels_idx, enlarged_batch_size * 2)
    masks = tf.one_hot(labels_idx, enlarged_batch_size)
  else:
    hidden1_large = hidden1
    hidden2_large = hidden2
    labels = tf.one_hot(tf.range(batch_size), batch_size * 2)
    masks = tf.one_hot(tf.range(batch_size), batch_size)

  logits_aa = tf.matmul(hidden1, hidden1_large, transpose_b=True) / temperature
  logits_aa = logits_aa - masks * LARGE_NUM
  logits_bb = tf.matmul(hidden2, hidden2_large, transpose_b=True) / temperature
  logits_bb = logits_bb - masks * LARGE_NUM
  logits_ab = tf.matmul(hidden1, hidden2_large, transpose_b=True) / temperature
  logits_ba = tf.matmul(hidden2, hidden1_large, transpose_b=True) / temperature

  loss_a = tfc.losses.softmax_cross_entropy(
      labels, tf.concat([logits_ab, logits_aa], 1), weights=weights)
  loss_b = tfc.losses.softmax_cross_entropy(
      labels, tf.concat([logits_ba, logits_bb], 1), weights=weights)
  loss = loss_a + loss_b

  return loss

### Github loss function by margokhokhlova

In [ ]:

@tf.function
def margokhokhlova_contrastive_loss(xi, xj,  tau=1, normalize=False):
        ''' this loss is the modified torch implementation by M Diephuis here: https://github.com/mdiephuis/SimCLR/
        the inputs:
        xi, xj: image features extracted from a batch of images 2N, composed of N matching paints
        tau: temperature parameter
        normalize: normalize or not. seem to not be very useful, so better to try without.
        '''

        x = tf.keras.backend.concatenate((xi, xj), axis=0)

        sim_mat = tf.keras.backend.dot(x, tf.keras.backend.transpose(x))

        if normalize:
            sim_mat_denom = tf.keras.backend.dot(tf.keras.backend.l2_normalize(x, axis=1).unsqueeze(1), tf.keras.backend.l2_normalize(x, axis=1).unsqueeze(1).T)
            sim_mat = sim_mat / sim_mat_denom.clamp(min=1e-16)

        sim_mat = tf.keras.backend.exp(sim_mat /tau)

        x = xi * xj
        if normalize:
            sim_mat_denom = tf.keras.backend.l2_normalize(xi, dim=1) * tf.keras.backend.l2_normalize(xj, axis=1)
            sim_match = tf.keras.backend.exp(tf.keras.backend.sum(x, axis=-1) / sim_mat_denom / tau)
        else:
            sim_match = tf.keras.backend.exp(tf.keras.backend.sum(x, axis=-1) / tau)

        sim_match = tf.keras.backend.concatenate((sim_match, sim_match), axis=0)

        norm_sum = tf.keras.backend.exp(tf.keras.backend.ones(tf.keras.backend.shape(x)[0]) / tau)

        return tf.math.reduce_mean(-tf.keras.backend.log(sim_match / (tf.keras.backend.sum(sim_mat, axis=-1) - norm_sum)), name='contrastive_loss')


##Data Augmentation

In the algorithm given in the SimCLR paper, each image in augmented twice and the two augmentation are put next to each other in the batch. The order is important in the loss function and since I think this loss function is the most problematic part I wanted to keep the order given in the algorithm. 

An other problem that became quickly apparent is that the dataset I use has 32x32 images, which gives pretty blurry images. Even for a human recognizing what's on the image is not always easy and some kind of augmentation like cropping, blur or too much color change can easily make the job needlessly hard for the neural network.

Lastly, I need to make the augmentation a @tf.function to reduce greatly the training time (from hours to minutes) which restricts me from doing some operations.

### Own Method

In [ ]:
@tf.function
def tensor_augment(x: tf.Tensor) -> tf.Tensor:
    """Color augmentation + random vertical flipt

    Args:
        x: Image

    Returns:
        Augmented image
    """
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    x = tf.image.random_flip_left_right(x)
    x = tf.clip_by_value(x,0,1)
    return x

@tf.function
def double_tensor_augment(image) :
    return tf.convert_to_tensor([tensor_augment(image),tensor_augment(image)])

#@tf.function
def augment_dataset(ds) :
  ds = ds.map(double_tensor_augment)
  out = ds.unbatch()
  return out

  

### FixMatch Method

In [ ]:
import random
import numpy as np
import scipy
from scipy import ndimage
from PIL import Image, ImageEnhance, ImageOps


import random
import numpy as np
import scipy
from scipy import ndimage
from PIL import Image, ImageEnhance, ImageOps

operations = {
    'ShearX': lambda img, magnitude: shear_x(img, magnitude),
    'ShearY': lambda img, magnitude: shear_y(img, magnitude),
    'TranslateX': lambda img, magnitude: translate_x(img, magnitude),
    'TranslateY': lambda img, magnitude: translate_y(img, magnitude),
    'Rotate': lambda img, magnitude: rotate(img, magnitude),
    'AutoContrast': lambda img, magnitude: auto_contrast(img, magnitude),
    'Invert': lambda img, magnitude: invert(img, magnitude),
    'Equalize': lambda img, magnitude: equalize(img, magnitude),
    'Solarize': lambda img, magnitude: solarize(img, magnitude),
    'Posterize': lambda img, magnitude: posterize(img, magnitude),
    'Contrast': lambda img, magnitude: contrast(img, magnitude),
    'Color': lambda img, magnitude: color(img, magnitude),
    'Brightness': lambda img, magnitude: brightness(img, magnitude),
    'Sharpness': lambda img, magnitude: sharpness(img, magnitude),
    'Cutout': lambda img, magnitude: cutout(img, magnitude),
}
def apply_policy(img, policy):
    if random.random() < policy[1]:
        img = operations[policy[0]](img, policy[2])
    if random.random() < policy[4]:
        img = operations[policy[3]](img, policy[5])
    # plt.imshow(img)
    return img


def transform_matrix_offset_center(matrix, x, y):
    o_x = float(x) / 2 + 0.5
    o_y = float(y) / 2 + 0.5
    offset_matrix = np.array([[1, 0, o_x], [0, 1, o_y], [0, 0, 1]])
    reset_matrix = np.array([[1, 0, -o_x], [0, 1, -o_y], [0, 0, 1]])
    transform_matrix = offset_matrix @ matrix @ reset_matrix
    return transform_matrix


def shear_x(img, magnitude):
    magnitudes = np.linspace(-0.3, 0.3, 11)

    transform_matrix = np.array([[1, random.uniform(magnitudes[magnitude], magnitudes[magnitude+1]), 0],
                                 [0, 1, 0],
                                 [0, 0, 1]])
    transform_matrix = transform_matrix_offset_center(transform_matrix, img.shape[0], img.shape[1])
    affine_matrix = transform_matrix[:2, :2]
    offset = transform_matrix[:2, 2]
    img = np.stack([ndimage.interpolation.affine_transform(
                    img[:, :, c],
                    affine_matrix,
                    offset) for c in range(img.shape[2])], axis=2)
    return img


def shear_y(img, magnitude):
    magnitudes = np.linspace(-0.3, 0.3, 11)

    transform_matrix = np.array([[1, 0, 0],
                                 [random.uniform(magnitudes[magnitude], magnitudes[magnitude+1]), 1, 0],
                                 [0, 0, 1]])
    transform_matrix = transform_matrix_offset_center(transform_matrix, img.shape[0], img.shape[1])
    affine_matrix = transform_matrix[:2, :2]
    offset = transform_matrix[:2, 2]
    img = np.stack([ndimage.interpolation.affine_transform(
                    img[:, :, c],
                    affine_matrix,
                    offset) for c in range(img.shape[2])], axis=2)
    return img


def translate_x(img, magnitude):
    magnitudes = np.linspace(-150/331, 150/331, 11)

    transform_matrix = np.array([[1, 0, 0],
                                 [0, 1, img.shape[1]*random.uniform(magnitudes[magnitude], magnitudes[magnitude+1])],
                                 [0, 0, 1]])
    transform_matrix = transform_matrix_offset_center(transform_matrix, img.shape[0], img.shape[1])
    affine_matrix = transform_matrix[:2, :2]
    offset = transform_matrix[:2, 2]
    img = np.stack([ndimage.interpolation.affine_transform(
                    img[:, :, c],
                    affine_matrix,
                    offset) for c in range(img.shape[2])], axis=2)
    return img


def translate_y(img, magnitude):
    magnitudes = np.linspace(-150/331, 150/331, 11)

    transform_matrix = np.array([[1, 0, img.shape[0]*random.uniform(magnitudes[magnitude], magnitudes[magnitude+1])],
                                 [0, 1, 0],
                                 [0, 0, 1]])
    transform_matrix = transform_matrix_offset_center(transform_matrix, img.shape[0], img.shape[1])
    affine_matrix = transform_matrix[:2, :2]
    offset = transform_matrix[:2, 2]
    img = np.stack([ndimage.interpolation.affine_transform(
                    img[:, :, c],
                    affine_matrix,
                    offset) for c in range(img.shape[2])], axis=2)
    return img


def rotate(img, magnitude):
    magnitudes = np.linspace(-30, 30, 11)

    theta = np.deg2rad(random.uniform(magnitudes[magnitude], magnitudes[magnitude+1]))
    transform_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                                 [np.sin(theta), np.cos(theta), 0],
                                 [0, 0, 1]])
    transform_matrix = transform_matrix_offset_center(transform_matrix, img.shape[0], img.shape[1])
    affine_matrix = transform_matrix[:2, :2]
    offset = transform_matrix[:2, 2]
    img = np.stack([ndimage.interpolation.affine_transform(
                    img[:, :, c],
                    affine_matrix,
                    offset) for c in range(img.shape[2])], axis=2)
    return img


def auto_contrast(img, magnitude):
    img = (img * 255).astype(np.uint8)
    img = Image.fromarray(img)
    img = ImageOps.autocontrast(img)
    img = np.array(img)
    return img


def invert(img, magnitude):
    img = (img * 255).astype(np.uint8)
    img = Image.fromarray(img)
    img = ImageOps.invert(img)
    img = np.array(img)
    return img


def equalize(img, magnitude):
    img = (img * 255).astype(np.uint8)
    img = Image.fromarray(img)
    img = ImageOps.equalize(img)
    img = np.array(img)
    return img


def solarize(img, magnitude):
    magnitudes = np.linspace(0, 256, 11)

    img = (img * 255).astype(np.uint8)
    img = Image.fromarray(img)
    img = ImageOps.solarize(img, random.uniform(magnitudes[magnitude], magnitudes[magnitude+1]))
    img = np.array(img)
    return img


def posterize(img, magnitude):
    magnitudes = np.linspace(4, 8, 11)

    img = (img * 255).astype(np.uint8)
    img = Image.fromarray(img)
    img = ImageOps.posterize(img, int(round(random.uniform(magnitudes[magnitude], magnitudes[magnitude+1]))))
    img = np.array(img)
    return img


def contrast(img, magnitude):
    magnitudes = np.linspace(0.1, 1.9, 11)

    img = (img * 255).astype(np.uint8)
    img = Image.fromarray(img)
    img = ImageEnhance.Contrast(img).enhance(random.uniform(magnitudes[magnitude], magnitudes[magnitude+1]))
    img = np.array(img)
    return img


def color(img, magnitude):
    magnitudes = np.linspace(0.1, 1.9, 11)

    img = (img * 255).astype(np.uint8)
    img = Image.fromarray(img)
    img = ImageEnhance.Color(img).enhance(random.uniform(magnitudes[magnitude], magnitudes[magnitude+1]))
    img = np.array(img)
    return img


def brightness(img, magnitude):
    magnitudes = np.linspace(0.1, 1.9, 11)

    img = (img * 255).astype(np.uint8)
    img = Image.fromarray(img)
    img = ImageEnhance.Brightness(img).enhance(random.uniform(magnitudes[magnitude], magnitudes[magnitude+1]))
    img = np.array(img)
    return img


def sharpness(img, magnitude):
    magnitudes = np.linspace(0.1, 1.9, 11)

    img = (img * 255).astype(np.uint8)
    img = Image.fromarray(img)
    img = ImageEnhance.Sharpness(img).enhance(random.uniform(magnitudes[magnitude], magnitudes[magnitude+1]))
    img = np.array(img)
    return img


def cutout(org_img, magnitude=None):
    magnitudes = np.linspace(0, 60/331, 11)

    img = np.copy(org_img)
    mask_val = img.mean()

    if magnitude is None:
        mask_size = 16
    else:
        mask_size = int(round(img.shape[0]*random.uniform(magnitudes[magnitude], magnitudes[magnitude+1])))
    top = np.random.randint(0 - mask_size//2, img.shape[0] - mask_size)
    left = np.random.randint(0 - mask_size//2, img.shape[1] - mask_size)
    bottom = top + mask_size
    right = left + mask_size

    if top < 0:
        top = 0
    if left < 0:
        left = 0

    img[top:bottom, left:right, :].fill(mask_val)

    return img

policies = [
              ['Invert', 0.1, 7, 'Contrast', 0.2, 6],
              ['Rotate', 0.7, 2, 'TranslateX', 0.3, 9],
              ['Sharpness', 0.8, 1, 'Sharpness', 0.9, 3],
              ['ShearY', 0.5, 8, 'TranslateY', 0.7, 9],
              ['AutoContrast', 0.5, 8, 'Equalize', 0.9, 2],
              ['ShearY', 0.2, 7, 'Posterize', 0.3, 7],
              ['Color', 0.4, 3, 'Brightness', 0.6, 7],
              ['Sharpness', 0.3, 9, 'Brightness', 0.7, 9],
              ['Equalize', 0.6, 5, 'Equalize', 0.5, 1],
              ['Contrast', 0.6, 7, 'Sharpness', 0.6, 5],
              ['Color', 0.7, 7, 'TranslateX', 0.5, 8],
              ['Equalize', 0.3, 7, 'AutoContrast', 0.4, 8],
              ['TranslateY', 0.4, 3, 'Sharpness', 0.2, 6],
              ['Brightness', 0.9, 6, 'Color', 0.2, 8],
              ['Solarize', 0.5, 2, 'Invert', 0, 0.3],
              ['Equalize', 0.2, 0, 'AutoContrast', 0.6, 0],
              ['Equalize', 0.2, 8, 'Equalize', 0.6, 4],
              ['Color', 0.9, 9, 'Equalize', 0.6, 6],
              ['AutoContrast', 0.8, 4, 'Solarize', 0.2, 8],
              ['Brightness', 0.1, 3, 'Color', 0.7, 0],
              ['Solarize', 0.4, 5, 'AutoContrast', 0.9, 3],
              ['TranslateY', 0.9, 9, 'TranslateY', 0.7, 9],
              ['AutoContrast', 0.9, 2, 'Solarize', 0.8, 3],
              ['Equalize', 0.8, 8, 'Invert', 0.1, 3],
              ['TranslateY', 0.7, 9, 'AutoContrast', 0.9, 1],
          ]
def augment(image):
  augmented_image = apply_policy(image, policies[random.randrange(len(policies))])
  #augmented_image = cutout(augmented_image)
  
  return augmented_image


def augment_batch_fixmatch(batch) :
  l = [ x for x in map(lambda x : [augment(image = np.float32(x)),augment(image = np.float32(x))] , batch)]
  out = []
  for x in l :
    out += x
  return out


##Custom model

### Definition

In [ ]:
class CustomModel(keras.Model):
  def __init__(self, body, head, Fixmatch = False, model_name = ""):
    super(CustomModel, self).__init__()
    self.body = body
    self.head = head
    self.Fixmatch = Fixmatch
    self.optimizer = -1
    now = datetime.now()
    init_time = now.astimezone(tz).strftime(" %H:%M")
    self.model_name = model_name + init_time


  def unfreeze_body_weights(self) :
    for layers in self.body.layers : # unfreeze weights
      layers.trainable= True

  def freeze_body_weights(self):
    for layers in self.body.layers : # unfreeze weights
      layers.trainable= False

  @tf.function
  def pretrain_step(self,batch_x, tot_loss, tot_loss_align = 0, additional_losses = False) :
    with tf.GradientTape() as tape: # GradientTape to record the operations and compute the gradiants
      y_pred = self(batch_x, training = True)
      N = len(y_pred)
      if additional_losses :
        loss_value, loss_align = contrastive_loss(y_pred,N,additional_losses = additional_losses)
        tot_loss_align += loss_align
      else :
        loss_value = contrastive_loss(y_pred,N, additional_losses = additional_losses)
        #loss_value = github_contrastive_loss(y_pred)
      tot_loss += loss_value
      loss_value += sum(self.losses)
    grads = tape.gradient(loss_value, self.trainable_weights) # computes the gradients
    self.optimizer.apply_gradients(zip(grads, self.trainable_weights))# Using the gradiants to optimize weights
    return tot_loss,tot_loss_align, grads, y_pred


  def custom_pretraining(self,dataset, epochs = 1,additional_losses = False, regular_save = False, 
                         verbose = 0, batch_size = 16, normalization = None):

    metric_contrastive_loss = []
    metric_loss_align = []

    if verbose == 1 :
      iterator1 = tqdm(range(1,epochs+1), desc="Contrasive training", unit = "epoch", position=0, leave=True)
    else :
      iterator1 = range(1,epochs+1)

    for epoch in iterator1 : # for each epoch
      shuffled_dataset = dataset.shuffle(1000)
      augmented_dataset = augment_dataset(shuffled_dataset)
      augmented_dataset = augmented_dataset.batch(batch_size*2)
      if normalization == "cifar10":
        augmented_dataset = augmented_dataset.map(normalize_cifar10_element)
      tot_loss = 0
      tot_loss_align = 0

      if verbose == 2 :
        iterator2 = enumerate(tqdm(augmented_dataset, desc=f"Epoch {epoch}", unit="batch",position=0, leave=True ))
      else : 
        iterator2 = enumerate(augmented_dataset) 

      for step, batch_x in iterator2  : # for each batch
        tot_loss,tot_loss_align, grads, y_pred = self.pretrain_step(batch_x,tot_loss, tot_loss_align, additional_losses)
      
      metric_contrastive_loss.append(tot_loss)
      metric_loss_align.append(tot_loss_align)

      if verbose == 2 :
        if additional_losses :
          print(f'Loss : {tot_loss} --- Loss Align : {tot_loss_align}')
        else :
          print(f'Loss : {tot_loss}')

      if regular_save & ((epoch)%10 == 0):
        tf.keras.models.save_model(self, PATH + str(f'{self.model_name} Pre-training -> epoch : {epoch}, optimizer : {self.optimizer}, FixMatch : {self.Fixmatch}'), overwrite=True)

    if additional_losses :
      return metric_contrastive_loss, metric_loss_align
    else :
      return metric_contrastive_loss

  def call(self,input_tensor, training = False):
    x = self.body(input_tensor, training = training)
    x = self.head(x, training = training)
    return x

  def predict_classes(self,input):
    return np.argmax(self.predict(input),axis = 1)

  def compile(self, optimizer='rmsprop', loss=None, metrics=None, loss_weights=None, weighted_metrics=None, run_eagerly=None, steps_per_execution=None, **kwargs):
    self.optimizer = optimizer
    return super().compile(optimizer, loss, metrics, loss_weights, weighted_metrics, run_eagerly, steps_per_execution, **kwargs)


# Evaluation

### Save & plot data

#### Utils

In [ ]:

def list_to_xlsx(x,y,doc, l, vertical = True, format = None) :
  length = 0
  if vertical :
    for i,data in enumerate(l) :
      doc.write(x+i,y, data, format)
      length = i
    return xl_rowcol_to_cell(x,y)+":"+xl_rowcol_to_cell(x+length,y)
  else :
    for i,data in enumerate(l) :
      doc.write(x,y+i, data, format)
    return xl_rowcol_to_cell(x,y)+":"+xl_rowcol_to_cell(x,y+length)

def xlsx_min(inter):
  return "=MIN("+inter+")"
def xlsx_max(inter):
  return "=MAX("+inter+")"

def xlsx_format_data(x,y,wb, ws,l, name, vertical = True, target = None) :
    min_max_format = wb.add_format({'bold': True, 'font_color': 'red'})
    target_format = wb.add_format({'bold': True, 'font_color': 'red', 'bg_color':'lime'})
    title_format = wb.add_format({'bold': True})
    if vertical :
      ws.write(x,y,name,title_format)
      inter_cl = list_to_xlsx(x+5,y,ws,l,vertical)
      if target == "max":
        ws.write_formula(x+3,y,xlsx_min(inter_cl),min_max_format)
        ws.write_formula(x+2,y,xlsx_max(inter_cl),target_format)
      elif target == "min":
        ws.write_formula(x+3,y,xlsx_min(inter_cl),target_format)
        ws.write_formula(x+2,y,xlsx_max(inter_cl),min_max_format)
      else :
        ws.write_formula(x+3,y,xlsx_min(inter_cl),min_max_format)
        ws.write_formula(x+2,y,xlsx_max(inter_cl),min_max_format)
    else :
      ws.write(x,y,name,title_format)
      inter_cl = list_to_xlsx(x,y+5,ws,l,vertical)
      if target == "max" :
        ws.write_formula(x,y+3,xlsx_min(inter_cl),min_max_format)
        ws.write_formula(x,y+2,xlsx_max(inter_cl),target_format)
      elif target == "min" :
        ws.write_formula(x,y+3,xlsx_min(inter_cl),target_format)
        ws.write_formula(x,y+2,xlsx_max(inter_cl),min_max_format)
      else :
        ws.write_formula(x,y+3,xlsx_min(inter_cl),min_max_format)
        ws.write_formula(x,y+2,xlsx_max(inter_cl),min_max_format)

#### Main

In [ ]:
def stories(story_metrics_cl, story_metrics_la, story_fit, story_ft, story_eval, step) :
  for i in range(len(story_metrics_cl)) :
    nb_epochs = step * i
    metrics_cl = story_metrics_cl[i]
    metrics_la = story_metrics_la[i]
    history_fit = story_fit[i]
    history_fine_tune = story_ft[i]
    evaluation = story_eval[i]

    plt.figure(figsize=(16,8))
    plt.title("Losses over contrastive training")

    plt.subplot(2,1,1)
    plt.plot(metrics_cl, label = "Contrastive Loss")
    plt.legend(loc='upper right')
    plt.xlabel("epoch")
    plt.ylabel("loss")

    plt.subplot(2,1,2)
    plt.plot(metrics_la, label = "Loss Align")
    plt.legend(loc='upper right')
    plt.xlabel("epoch")
    plt.ylabel("loss")

    plt.savefig(PATH+f"epochs{nb_epochs}_contrastive_training.png")
    plt.show()


    plt.figure(figsize=(16,8))
    plt.title("Losses & Accuracies over fitting")

    plt.subplot(2,1,1)
    plt.plot(history_fit.history['accuracy'], label = "training accuracy")
    plt.plot(history_fit.history['val_accuracy'], label = "validation accuracy")
    plt.legend(loc='upper right')
    plt.xlabel("epoch")
    plt.ylabel("accuracy")

    plt.subplot(2,1,2)
    plt.plot(history_fit.history['loss'], label = "training loss")
    plt.plot(history_fit.history['val_loss'], label = "validation loss")
    plt.legend(loc='upper right')
    plt.xlabel("epoch")
    plt.ylabel("loss")

    plt.savefig(PATH+f"_epochs{nb_epochs}_fitting.png")
    plt.show()


    plt.figure(figsize=(16,8))
    plt.title("Losses & Accuracies over fine tuning")

    plt.subplot(2,1,1)
    plt.plot(history_fine_tune.history['accuracy'], label = "training accuracy")
    plt.plot(history_fine_tune.history['val_accuracy'], label = "validation accuracy")
    plt.plot([evaluation[1] for  _ in range(len(history_fine_tune.history['val_accuracy']))], label = "test accuracy")
    plt.legend(loc='upper right')
    plt.xlabel("epoch")
    plt.ylabel("accuracy")

    plt.subplot(2,1,2)
    plt.plot(history_fine_tune.history['loss'], label = "training loss")
    plt.plot(history_fine_tune.history['val_loss'], label = "validation loss")
    plt.plot([evaluation[0] for  _ in range(len(history_fine_tune.history['val_loss']))], label ="test loss")
    plt.legend(loc='upper right')
    plt.xlabel("epoch")
    plt.ylabel("loss")

    plt.savefig(PATH+f"_epochs{nb_epochs}_fine_tuning.png")
    plt.show()


def save_stories_xlsx(story_metrics_cl, story_metrics_la, story_fit, story_ft, story_eval, step, name) :
  
  writer = pd.ExcelWriter(name+'.xlsx', engine='xlsxwriter',options={'nan_inf_to_errors': True})
  workbook = writer.book
  target_format = workbook.add_format({'bold': True, 'font_color': 'red'})
  title_format = workbook.add_format({'bold': True})

  for i in range(len(story_metrics_cl)) :
    nb_epochs = step * (i+1)
    metrics_cl = story_metrics_cl[i]
    metrics_la = story_metrics_la[i]
    history_fit = story_fit[i]
    history_fine_tune = story_ft[i]
    evaluation = story_eval[i]
    worksheet = workbook.add_worksheet(f"With {nb_epochs} epochs")
    #worksheet.write(0,1, "Target value",title_format)
    worksheet.write(4,0, "min",title_format)
    worksheet.write(5,0, "max",title_format)
    
    worksheet.set_column(2, 2, 15)
    worksheet.set_column(4, 5, 15)
    worksheet.set_column(7, 10, 15)
    worksheet.set_column(12, 15, 15)

    xlsx_format_data(2,4,workbook,worksheet, metrics_cl, "Contrastive loss", target = "min")
    xlsx_format_data(2,5,workbook,worksheet, metrics_la, "Loss align", target = "min")

    xlsx_format_data(2,7,workbook,worksheet, history_fit.history["loss"], "Fit - Loss", target = "min")
    xlsx_format_data(2,8,workbook,worksheet, history_fit.history["val_loss"], "Fit - Validation loss", target = "min")
    xlsx_format_data(2,9,workbook,worksheet, history_fit.history["accuracy"], "Fit - Accuracy", target = "max")
    xlsx_format_data(2,10,workbook,worksheet, history_fit.history["val_accuracy"], "Fit - Validation accuracy", target = "max")

    xlsx_format_data(2,12,workbook,worksheet, history_fine_tune.history["loss"], "Fine tune - Loss", target = "min")
    xlsx_format_data(2,13,workbook,worksheet, history_fine_tune.history["val_loss"], "Fine tune - Validation loss", target = "min")
    xlsx_format_data(2,14,workbook,worksheet, history_fine_tune.history["accuracy"], "Fine tune - Accuracy", target = "max")
    xlsx_format_data(2,15,workbook,worksheet, history_fine_tune.history["val_accuracy"], "Fine tune - Validation accuracy", target = "max")

    xlsx_format_data(2,2,workbook,worksheet, evaluation, "Test Loss & Accuracy")

  writer.save()




## General evaluation


In [ ]:
def run_evaluation(dic):
  now = datetime.now()
  day = now.astimezone(tz).strftime("%d-%m-%Y")
  date_start = now.astimezone(tz).strftime("%d-%m-%Y %H:%M:%S")
  print(f'Début de l\'import et du traitement des données : {date_start}')
  unlabeled_ds, train_ds, test_ds, val_ds = generate_datasets()

  now = datetime.now()
  date = now.astimezone(tz).strftime("%d-%m-%Y %H:%M:%S")
  print(f'Début du protocole : {date}')


  story_metrics_cl = []
  story_metrics_la = []
  story_fit = []
  story_ft = []
  story_eval = []
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=dic["cb_patience"], mode = "max")

  for i in range(dic["contrastive_training_steps"]) :
    nb_epochs = dic["regular_save"]*(i+1)
    now = datetime.now()
    date = now.astimezone(tz).strftime("%d-%m-%Y %H:%M:%S")
    print(f'Début du test avec l\'epochs numero {nb_epochs}, {date}')

    classifier = build_classifier()
    projection_head = build_projection_head()
    if i == 0 :
      if dic["body"] == "widenet" :
        body = build_WideNet()
      elif dic["body"] == "resnet" :
        body = build_resnet()
      elif dic["body"] == "conv" :
        body = build_simple_conv()
        
      model = CustomModel(body,projection_head)
      model.build([None,32,32,3])
      model.compile(optimizer= keras.optimizers.Adam(learning_rate=dic["contrastive_lr"]),
                  loss = keras.losses.categorical_crossentropy,
                  metrics = ['accuracy'])
    else :
      model.load_weights("temp")

    model.unfreeze_body_weights()
    metrics_cl, metrics_la = model.custom_pretraining(unlabeled_ds, epochs=dic["regular_save"], batch_size = dic["contrastive_batch_size"], additional_losses = True, verbose = 2, normalization="cifar10")
    story_metrics_cl.append(metrics_cl)
    story_metrics_la.append(metrics_la)

    model.save_weights("temp")

    model2 = CustomModel(body, classifier)
    model2.build([None,32,32,3])
    model2.compile(optimizer= keras.optimizers.Adam(learning_rate=dic["training_lr"]),
                  loss = keras.losses.categorical_crossentropy,
                  metrics = ['accuracy'])
    if dic["training_freeze"] :
      model2.freeze_body_weights()
    else :
      model2.unfreeze_body_weights()
    history_fit = model2.fit(train_ds, validation_data = val_ds, batch_size=dic["training_batch_size"], epochs = dic["training_epochs"], callbacks=[callback], verbose = 1)  
    story_fit.append(history_fit)

    model3 = CustomModel(body, classifier)
    model3.build([None,32,32,3])
    model3.compile(optimizer= keras.optimizers.Adam(learning_rate=dic["finetuning_lr"]),
                  loss = keras.losses.categorical_crossentropy,
                  metrics = ['accuracy'])
    if dic["finetuning_freeze"] :
      model3.freeze_body_weights()
    else :
      model3.unfreeze_body_weights()
    history_fine_tune = model3.fit(train_ds, validation_data = val_ds, batch_size=dic["finetuning_batch_size"], epochs = dic["finetuning_epochs"], verbose = 1)  
    story_ft.append(history_fine_tune)

    evaluation = model3.evaluate(test_ds,batch_size=dic["evaluation_batch_size"])
    story_eval.append(evaluation)

  now = datetime.now()
  date_end = now.astimezone(tz).strftime("%d-%m-%Y %H:%M:%S")
  print(f'Fin du protocole : {date_end}')
  return story_metrics_cl, story_metrics_la, story_fit, story_ft, story_eval, date_start, date_end

In [ ]:
Training_dic = {
    "contrastive_training_steps" : 10,
    "regular_save" : 2,
    "training_epochs" : 10,
    "finetuning_epochs" : 10,
    "body" : 'resnet', # resnet / widenet / conv
    "cb_patience" : 5,
    "training_freeze" : False,
    "finetuning_freeze" : False,
    "contrastive_batch_size": 64,
    "training_batch_size" : 32,
    "finetuning_batch_size" : 32,
    "evaluation_batch_size" : 64,
    "contrastive_lr" : 0.001,
    "training_lr" : 0.001,
    "finetuning_lr" : 0.000001
}

story_metrics_cl, story_metrics_la, story_fit, story_ft, story_eval, date_start, date_end = run_evaluation(Training_dic)

Début de l'import et du traitement des données : 07-11-2021 04:45:26


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
160/160 [==============================] - 24s 108ms/step - loss: 2.3135 - accuracy: 0.2233 - val_loss: 2.8735 - val_accuracy: 0.2460
Epoch 2/10
160/160 [==============================] - 16s 100ms/step - loss: 2.1129 - accuracy: 0.2731 - val_loss: 1.9894 - val_accuracy: 0.3150
Epoch 3/10
160/160 [==============================] - 16s 100ms/step - loss: 2.0387 - accuracy: 0.3117 - val_loss: 3.1107 - val_accuracy: 0.3230
Epoch 4/10
160/160 [==============================] - 16s 100ms/step - loss: 1.9284 - accuracy: 0.3374 - val_loss: 1.9202 - val_accuracy: 0.3590
Epoch 5/10
160/160 [==============================] - 16s 101ms/step - loss: 1.7698 - ac

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
160/160 [==============================] - ETA: 0s - loss: 1.3171 - accuracy: 0.5209WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7ff26a557830> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
160/160 [==============================] - 23s 108ms/step - loss: 1.3171 - accuracy: 0.5209 - val_loss: 1.4990 - val_accuracy: 0.4520
Epoch 2/10
160/160 [==============================] - 16s 101ms/step - loss: 1.3111 - accuracy: 0.5236 - val_loss: 1.5102 - val_accuracy: 0.4540
Epoch 3/10
160/160 [==============================] - 16s 101ms/step - loss: 1.3059 - accuracy: 0.5260 - val_loss: 1.5118 - val_accuracy: 0.4560
Epoch 4/10
160/160 [==============================] - 16s 101ms/step - loss: 1.3009 - accuracy: 0.5284 - val_loss: 1.5110 - val_accuracy: 0.4560
Epoch 5/10
160/160 [==============================] - 16s 100ms/step - loss: 1.2957 - ac

In [ ]:
save_stories_xlsx(story_metrics_cl, story_metrics_la, story_fit, story_ft, story_eval, 2, date_start)

NameError: ignored

# Tests


## Unit tests

### Testes tf.data.dataset


#### Few tests ot show how tfds.load works with split

In [ ]:
#train_ds, test_ds = tfds.load("cifar10", split=['train','test'], as_supervised=True, batch_size=10)
#print(len(train_ds))
#print(len(test_ds))

#unlabeled_ds = tfds.load("cifar10", split='train', as_supervised=True, batch_size=10)
#labeled_ds = tfds.load("cifar10", split='test',as_supervised=True, batch_size=10)
#print(len(unlabeled_ds))
#print(len(labeled_ds))

#train_ds, test_ds, validation_ds = tfds.load("cifar10", split=['test[0%:80%]','test[80%:90%]','test[90%:100%]'],as_supervised=True, batch_size=10)
#print(len(train_ds))
#print(len(test_ds))
#print(len(validation_ds))

#### Few tests to show the structure of a dataset

In [ ]:
train_ds, test_ds = tfds.load("cifar10", split=['train','test'], as_supervised=True)

In [ ]:
train_ds.shuffle(buffer_size=100)
train_ds = train_ds.take(1)
x = train_ds.map(get_x)
for image in x :
  # We go over all the batch but there is only one thanks to take()
  print(image)

In [ ]:
xtds = train_ds.map(get_x)
#print(xtds)
#print(ytds)
xtds = xtds.map(format_batch)
ytds = train_ds.map(get_y)

#print(ytds)
#plot_batch(ytds)



#### Test of the dataset augmentation

In [ ]:
def extract_batch(ds) :
  for batch in ds :
    return batch
    
unlabeled_ds, train_ds, test_ds, val_ds = generate_datasets()
shuffled_dataset = unlabeled_ds.shuffle(1000)
augmented_dataset = augment_dataset(shuffled_dataset)
augmented_dataset = augmented_dataset.batch(10)
x = augmented_dataset.take(1)
print(x)
x = extract_batch(x)
plot_batch(x)


###Tests of contrastive loss


The contrastive loss is, I think, the main difficulty of this project and I probably still to this day didn't make one that works as I want to and is the reason why the performances I get aren't so great.

Difficulty 1 : The loss function is quite a complicated math function and test the loss function by hand in a unit test is not really easy nor explicit.

Difficulty 2 : The loss function needs to be a @tf.function to drasticaly cut down the training time. This mean I can't do any operation in the loss function but I am limited to a small set of operations (ex: the module tf.math) to reproduce the formula and making it harder to understand what other implementations do.

Difficulty 3 : I found several loss function that claim to be implementation of the SimCLR paper (including the one given on the github repository of the SimCLR paper writers) but none of them seem to match the mathematical formula given in the paper. Probably either because of a mistake or a different implementation somewhere else (of the datasets for instance) changing the loss function.

Test 1 : I tested the variations of the loss functions on small test cases, it seems that my function has the right variations.

Test 2 : I tested the corelation between some of the loss funciton I found and my own and I didn't find any corelation between them so at most one of them is correct

Conclusion : I still don't know if my custom loss function is correct but it is the best i have for now.

In [ ]:

proj_test1 = tf.constant([[0,0,0,0.9],[0,0,0,0.88],[0.5,0.5,0,0],[0.89,0.1,0,0]])
proj_test2 = tf.constant([[0,0,0,0.9],[0,0,0,0.88],[0.7,0.3,0,0],[0.89,0.1,0,0]])
print(contrastive_loss(proj_test1,indices = get_indices_l(2)))
print(contrastive_loss(proj_test2,indices = get_indices_l(2)))

In [ ]:

unlabeled_ds, train_ds, test_ds, val_ds = generate_datasets()
l1 = []
l2 = []
for i in trange(200) :
  shuffled_dataset = unlabeled_ds.shuffle(50000)
  augmented_dataset = augment_dataset(shuffled_dataset)
  augmented_dataset = augmented_dataset.batch(10)
  x = augmented_dataset.take(1)

  body = build_simple_conv()
  head = build_projection_head()
  model = CustomModel(body,head)
  model.build([None,32,32,3]) 
  model.compile(optimizer= keras.optimizers.Adam(),
                  loss = keras.losses.categorical_crossentropy,
                metrics = ['accuracy'])

  for batch in x :
    y = model(batch)
    #print(y)  #  Just one

  l1.append(float(contrastive_loss(y, 10)))
  l2.append(float(github_contrastive_loss(y)))


In [ ]:
plt.figure()
plt.plot(l1,l2)
plt.show()

### Test of freeze & unfreeze

In [ ]:
def count_params(model):
  nb_non_trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.non_trainable_weights])
  nb_trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.trainable_weights])
  return nb_non_trainable_params, nb_trainable_params

In [ ]:
head = build_classifier()
body = build_simple_conv()
model = CustomModel(body, head)
model.build([None,32,32,3])
model.compile(optimizer= keras.optimizers.Adam(),
                loss = keras.losses.categorical_crossentropy,
                metrics = ['accuracy'])

nb_nt, nb_t = count_params(model)
print(nb_nt)
print(nb_t)
model.summary()

model.freeze_body_weights()

nb_nt, nb_t = count_params(model)
print(nb_nt)
print(nb_t)
model.summary()

model.unfreeze_body_weights()

nb_nt, nb_t = count_params(model)
print(nb_nt)
print(nb_t)
model.summary()

### Tests of different bodies


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, mode = "max")
body = build_simple_conv()
classifier = build_classifier()
model = CustomModel(body,classifier)
model.build([None,32,32,3]) 
model.compile(optimizer= keras.optimizers.Adam(),
                loss = keras.losses.categorical_crossentropy,
               metrics = ['accuracy'])
model.freeze_body_weights()
history = model.fit(train_ds, validation_data=val_ds, batch_size=32, epochs = 100, callbacks=[callback])
eval = model.evaluate(test_ds, batch_size=128)

In [ ]:
train_ds = tfds.load(bank, split='train', as_supervised=True)
train_ds.map(format_)
train_ds, test_ds, validation_ds = tfds.load(bank, split=['test[0%:80%]','test[80%:90%]','test[90%:100%]'],as_supervised=True, batch_size=batch_size)


callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, mode = "max")
body = build_resnet()
classifier = build_classifier()
model = CustomModel(body,classifier)
model.build([None,32,32,3]) 
model.compile(optimizer= keras.optimizers.Adam(),
                loss = keras.losses.categorical_crossentropy,
               metrics = ['accuracy'])
history = model.fit(train_ds, validation_data=val_ds, batch_size=32, epochs = 100, callbacks=[callback])
eval = model.evaluate(test_ds, batch_size=128)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, mode = "max")
body = build_WideNet()
classifier = build_classifier()
model = CustomModel(body,classifier)
model.compile(optimizer= keras.optimizers.Adam(),
                loss = keras.losses.categorical_crossentropy,
               metrics = ['accuracy'])
history = model.fit(train_ds, validation_data=val_ds, batch_size=32, epochs = 100, callbacks=[callback])
eval = model.evaluate(test_ds, batch_size=128)

## Tests of WideResNet


In [ ]:

# Load the datasets

train_ds, test_ds = tfds.load('cifar10', split=['train','test'],as_supervised=True, batch_size=batch_size)

# Format the labeled datasets
train_ds = train_ds.map(format_image,  num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(format_image,  num_parallel_calls=AUTOTUNE)

# Normalize the labeled datasets
train_ds = train_ds.map(normalize_cifar10_element,  num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(normalize_cifar10_element,  num_parallel_calls=AUTOTUNE)

# Format the labeled datasets
train_ds = train_ds.map(lambda x,y: from_int_to_categorical(x,y,num_classes))
test_ds = test_ds.map(lambda x,y: from_int_to_categorical(x,y,num_classes))

# Prefecth the labeled datasets
train_ds = train_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)


callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, mode = "max")

body = build_WideNet()
classifier = build_classifier()
model = CustomModel(body,classifier)
model.build([None,32,32,3])
model.compile(optimizer= keras.optimizers.Adam(learning_rate=0.001),
            loss = keras.losses.categorical_crossentropy,
            metrics = ['accuracy'])
    
history_fit = model.fit(train_ds, validation_data = test_ds, batch_size=32 ,epochs = 100, callbacks=[callback], verbose = 1)  


In [ ]:
train_ds, test_ds, validation_ds = tfds.load('cifar10', split=['test[0%:80%]','test[80%:90%]','test[90%:100%]'],as_supervised=True, batch_size=batch_size)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, mode = "max")
classifier = build_classifier()
model = CustomModel(body,classifier)
model.build([None,32,32,3])
model.compile(optimizer= keras.optimizers.Adam(learning_rate=0.0005),
            loss = keras.losses.sparse_categorical_crossentropy,
            metrics = ['accuracy'])
model.freeze_body_weights()
history_fit = model.fit(train_ds, validation_data=validation_ds,batch_size=32 ,epochs = 100, callbacks=[callback], verbose = 1)  
model.evaluate(test_ds)